<center> <h1>Segunda entrega del proyecto</h1> </center>
<center> <h3>Yeison Cuervo Basurto cc.1024602364</h3> </center>
<center> <h3>Matemáticas Discretas II</h3> </center>
<center> <h3>Universidad Nacional de Colombia</h3> </center>
<center> <h3>04/12/2020</h3> </center>

## Introducción

Todo tipo de trasmisión de códigos que involucre en algún momento la transmisión de un medio a otro puede presentar errores, por ejemplo, los códigos de barras o códigos de tarjetas de credito, estos errores representan problemas para muchas empresas, ya que si estos errores no existieran algunas empresas podrían abaratar costes en la impresión de códigos de barras.

Para tratar este problema se usan los digitos de confirmación los cuales se basan en el uso del algoritmo de luhn sin embargo estos tampoco garantizan que si los datos son recibidos entonces no se presentó ningún error en la trascripción de los datos, lo cual hace necesario volver a leer el código o si tiene algún pequeño error de impresión entonces se hace necesario reeimprimir esa etiqueta en cuestión. 

Mas concretamente los errores que este sistema no soporta son algunos errores de transposición y algunos errores de cambio de dígito individuales. Se busca en este proyecto entender el funcionamiento de este algoritmo, entender la naturaleza de estos errores y comparar el algoritmo con otros métotos para la detección de errores.

## Materiales y métodos

Los principales métodos que se utilizan serán los que me permiten calular un digito de comprobación para un vector de números y para la creación de un codigo de hamming que permita según el caso recibir y corregir un cierto número binario, este caso es interesante ya que se pueden tratar errores comunes que se encuentran en los distintos protocolos usados para la notación de códigos de barras o numeros de tarjetas de crédito.

### Algoritmo de luhn

Antes de poner el código, se hace la explicación mátemática, primero es necesario tener la siguiente notación útil en cuenta, suponga un vector en el cual cada elemento está en base b y tiene una magnitud p, entonces ese vector se puede denotar $\mathbb{Z}^{k}_{p}$.

Teniendo en cuenta lo anterior entonces supongamos un vector binario $\mathbb{v} = [1001x]$ en $\mathbb{Z}^{5}_{2}$, este vector representa el número 9, una de las estrategias usadas en la detección de errores es la estrategia del dígito de paridad, que consiste en verificar que la suma de todos los digitos de $\vec{v}$ es congruente con $0 \text{ mod } 2$, o lo que es lo mismo, comprobar que  $\vec{v}\cdot\vec{1} = \vec{0}$, teniendo en cuenta que se está trabajando en base 2, para este caso particular, se tiene que  $x = 0$.

#### Descripción matematica

Lo anterior se puede generalizar y se le puede añadir otras comprobaciones para detectar mas errores, por ejemplo supongamos un vector $\vec{u} = [56489d]$ en $\mathbb{Z}^{6}_{10}$ para este caso se tiene lo mismo, el vector debe cumplir que $\vec{u}\cdot\vec{1} = \vec{0}$ teniendo en cuenta que trabajamos en base 10, Esta estrategía presenta errores en cuanto a su diseño pues supongamos que originalmente mandamos el mensaje $[564897d]$ pero llega  $[546897d]$ en ese caso no se detectaría el error y se entendería que el mensaje fue leido correctamente, eso sería malo pues podría en caso de ser un código de barras recibir un codigo diferente al necesario, para esto se usa un digito de verificación, que comprueba algo distinto al codigo de paridad.

Hay varias formas de plantear el código de verificación, la idea principal es cambiar el vector $\vec{1}$ por algún vector conveniente $\vec{a}$ este vector nos debe ayudar a comprobar este hecho, por lo general se usa un vector alternante que cambie de a dos valores que consigan que cuando se permuten los conponentes adyacentes del vector, entonces el producto punto de los vectores $\vec{1}$ y $\vec{a}$ sean distintos. Se pueden tomar los valores 1 y 3 que cumplen esto hasta cierto punto, es más, estos valores son usados para varios estandares de codigos de control de errores, como los códigos de barras EAN y UPC.

En este caso, este código presenta una falla, ya que para que dos códigos de los cuales uno tiene dos valores adyacentes transpuestos puedan tener el mismo codigo de verificación solo hace falta que se cumpla que el producto $\vec{u}$ y $\vec{a}$, sea el mismo para algún vector $\vec{u}$ que solo tenga un componente adyacene transpuesto con respecto a otro, esto se cumple si para cualesquiera dos componente $a$ y $b$ adyacentes, se cumple que:

\begin{equation*}
  \begin{array}{c}
 	3a + 3b = a+b\\
    2a + 2b = 0\\
  \end{array}
\end{equation*}

y para un caso particular se tiene que  $ 2(5) + 2(0) = 0$, esto tiene sentido si se tiene en cuenta que estamos trabajando bajo modulo 10. 

Se puede usar el vector $\vec{a} = [131313\cdots 1]$ que arregla este problema casi por completo, por lo tanto:

\begin{equation*}
  \begin{array}{c}
 	\vec{u} \cdot \vec{a} = [5,(3\times 6),4,(3\times 8),9,(3\times 7),d] = 1+d = 0\\
    1 + d = 0\\
    d = 9
  \end{array}
\end{equation*}

El codigo en python que me genera el codigo de verificación se muestra a continuación:

In [88]:
"""
Codigo que encuentra el codigo de verificación para el vector de control especifico ingresado, se ingresa hasta antes del
codigo de verificación. 
"""

def codigoVer(codi):
    
    codst = list(map(int, str(codi)))
    
    cod = []
    for i in range(len(codst)):
        cod.append(int(codst[i])) 
    
    suma = 0
    for i in range(1,len(cod),2):
        cod[i] = (cod[i] * 3) % 10
        
    for i in range(0,len(cod)):
        
        suma = suma + cod[i]
    
    return "El digito de control es: "+ str(10-(suma % 10))
        

In [211]:
# Ejemplo: Se calcula el codigo de verificación del caso mostrado

print(codigoVer("564897"))

El digito de control es: 9


#### Principal problema de algoritmo de luhn

El principal problema de este algoritmo es que se pueden encontrar dos códigos distintos que difieren de un digito adyacente que general el mismo codigo de verificación, a continuación se muestra probando caso por caso de un código particular, para que valores trasnpuestos se puede generar el mismo código de verificación, en los comentarios del código se aprecia la explicación.

In [213]:
"""
Ejemplo 2:

Esta parte muestra para que valores se pueden generar el mismo código de verficiación para codigos en los que fue cambiado
un componente adyacente
"""


for i in range(10):
    for j in range(10): # for que generan numeros de 0 a 9
        
        # Esta sección del código solo se ejecuta si el codigo generado es el mismo y los numeros i y j no son iguales
        
        if codigoVer(str(i)+str(j)+"492702090169") == codigoVer(str(j)+str(i)+"492702090169") and i != j:
            
            # permuta los primeros 2 elementos del código de manera que se intercambién dos digitos adyacentes
            print("")
            print(str(i)+"----"+str(j))
            print(codigoVer(str(i)+str(j)+"492702090169"))
            print(str(j)+"----"+str(i))
            print(codigoVer(str(j)+str(i)+"492702090169"))
            
 



0----5
El digito de control es: 2
5----0
El digito de control es: 2

1----6
El digito de control es: 8
6----1
El digito de control es: 8

2----7
El digito de control es: 4
7----2
El digito de control es: 4

3----8
El digito de control es: 10
8----3
El digito de control es: 10

4----9
El digito de control es: 6
9----4
El digito de control es: 6

5----0
El digito de control es: 2
0----5
El digito de control es: 2

6----1
El digito de control es: 8
1----6
El digito de control es: 8

7----2
El digito de control es: 4
2----7
El digito de control es: 4

8----3
El digito de control es: 10
3----8
El digito de control es: 10

9----4
El digito de control es: 6
4----9
El digito de control es: 6


A continuación se explica el codigo de hamming que permite detectar errores y corregirlos en el mismo punto.

### Codigo de Hamming

Supongamos que deseamos que cuando se reciba un código de barras que tiene algún error en algún digito del código, que el error sea detectado y corregido al mismo tiempo, esto ahorraría tiempo en las lectura de códigos de barras y se podría ahorrar en la calidad de la impresión.

Para hacer algo como esto se usa el codigo hamming que permite detectar los errores de transposición adyacende y de cambio de digitos, este código sirve solo para vectores binarios, pero esto no impide su uso a este tipo de usos. Este código connsiste en generar un sistema de ecuaciones lineales para un código es decir que todos los codigos deben cumplirlo.

#### Descripción matemática

Suponga que necesita 16 elementos distintos de un conjunto de productos, se supone un conjunto pequeño para ilustrar su funcionamiento, pero se pueden hacer mucho mas amplios los calculos, se tienen entonces que para representar esos 16 elementos se necesitan cuatro dígitos distintos se pueden escribir como  $a_1a_2a_3a_4$, sin embargo para verificar los direfentes tipos de errores, se necesitan añadir mas digitos de control $b_1b_2b_3$, estos digitos de control tienen que cumplir ciertas propiedades, como en los casos anteiores, la suma de todos los digitos $ \text{mod } 2$ debe ser congruente con $ 0 \text{ mod } 2$.

Para comprobar el dígito de control se hace lo siguiente, se plantea el siguiente sistema de ecuaciones:

\begin{equation*}
  \begin{array}{c}
 	b_1 = a_4 + 0\cdot a_3  +a_2 + a_1\\
    b_2 = a_4 + a_3 + 0 \cdot a_2 + a_1\\
    b_3 = a_4 + a_3 + a_2 + 0 \cdot a_1\\
    \end{array}
\end{equation*}

este sistema nos comprueba que si se cabia algún valor adyacen por alún otro entonces el sistema se ve alterado, por ejemplo si se cambiar $a_4$ por $a_3$ entonces ocurriría que un $a_4$ quedaría multiplicado por cero y $a_3$ no, esto nos mostraría que ocurrio no solo un error sino que sabríamos que valor de $b$ es el que está fallando pues la respectiva ecuación no se cumpliría.

Esto como todo sistema de ecuaciones se puede convertir en una matriz si se hace el siguiente cambio:

\begin{equation*}
  \begin{array}{c}
 	a_4 + 0\cdot a_3  +a_2 + a_1 + b_1 +  0 \cdot b_2 +  0 \cdot b_3= 0\\
    a_4 + a_3 + 0 \cdot a_2 + a_1 +  0 \cdot b_1 + b_2 +  0 \cdot b_3 =0\\
    a_4 + a_3 + a_2 + 0 \cdot a_1 +  0 \cdot b_1 +  0 \cdot b_3 + b_3 =0\\
    \end{array}
\end{equation*}

entonces se genera la siguiente matriz:

\begin{equation*}
  A = \left( \begin{array}{ccccccc}
 	1 & 0 & 1 & 1 & 1 & 0 & 0\\
    1 & 1 & 0 & 1 & 0 & 1 & 0\\
    1 & 1 & 1 & 0 & 0 & 0 & 1\\
    \end{array} \right)
\end{equation*}

Esta matriz se usa para comprobar si cierto vector recibido cumple con las condiciones preestablecidas, y además si cierto vector presenta algún error, el resultado nos ayuda a ver donde ocurrió el error, por ejemplo vamos a suponer que se envía el siguiente vector en $\mathbb{Z}^{7}_{2}$ (que representaría el digito 9) $\vec{w} = [1001001]$ con el cual se obtiene $A\vec{w}^{T} = 0$, y si se cambia algún valor para este dígito el vector resultante nos diría, según la columna resultante de A en que vector ocurrió el error. Por ejemplo si se escribe $\vec{w}= [1011001]$ entonces el resultado es $[101]$ que es la tercera columna de $A$, lo que nos indica que el error ocurrió en el tercer componente.

\begin{equation*}
  A = \left( \begin{array}{ccccccc}
 	1 & 0 & 1 & 1 & 1 & 0 & 0\\
    1 & 1 & 0 & 1 & 0 & 1 & 0\\
    1 & 1 & 1 & 0 & 0 & 0 & 1\\
    \end{array} \right)
    \left( \begin{array}{c}
 	1 \\
    0 \\
    1 \\
    1 \\
    0 \\
    0 \\
    1 \\
    \end{array} \right) = 
    \left( \begin{array}{c}
 	1 \\
    0 \\
    1 \\
\end{array} \right)
\end{equation*}



A continuación se muestra un código que permite generar estos códigos de verificación para una palabra de 64 bits, para estas paralabras se necesitan siete digitos de confirmación. 

In [94]:
"""
Funcion que permite generar los dígitos de confirmación para palabras de hasta 64 bits 
"""

def codigohamming(codi):
    
    num = int(codi)
    codbin = []
      
    #Esta parte convierte el numero a binario
    while num >= 1:
        codbin.append(num % 2)
        num = num // 2
    codbin.reverse()
    
    
    #Esta parte selecciona el número de digitos de comrobación según el caso 
    tam = len(codbin)
    if(tam > 64):
        return "Muy Largo"
    elif tam <= 64 and tam > 32:
        bits_agregados = 7
    elif tam <= 32 and tam > 16:
        bits_agregados = 6
    elif tam <= 16 and tam > 8:
        bits_agregados = 5
    elif tam <= 8 and tam > 4:
        bits_agregados = 4
    elif tam <= 4 and tam > 2:
        bits_agregados = 3
    
    #Esta parte genera los digitos de comprobacion
    for j in range(bits_agregados):  
        suma = 0
        for k in range(tam):
            if k != j+1:
                suma = suma + codbin[k]
            
        codbin.append(suma % 2)
        
    
    return([codbin,tam])

In [95]:
"""
Codigo para confirmar si un codigo recibido es correcto
"""

def retornaConfirmacio(vector):
    res = []
    
    #Esta parte multiplica el vector por la matriz de comprobación adecuada
    for j in range(len(vector[0])-vector[1]):
        suma = 0
        for k in range(vector[1]):
            if k != j+1:
                suma = suma + vector[0][k]
        
        for w in range(len(vector[0])-vector[1]):
            if w == j:
                suma = suma + vector[0][vector[1]+w]
        
        res.append(suma % 2)
        
    return res
            


#### Problema código hamming

El único problema que se le puede achacar es que a diferencia del algoritmo de luhn la cantidad de calculos que hay que realizar es mucho mayor, mientras que el algoritmo de luhn es de orden lineal pues hay que hacer un producto y una suma de n elementos, los cuales suelen ser pocos mientras que en el algoritmo de luhn hay que pasar el número a binario y luego hacer las operaciones con esos maximos 64 + 7 digitos contando los digitos de confirmación, estas operaciones involucran un producto de n por n digitos así que para generar este código se requiere de un algorimo orden $n^{2}$.

Otro inconveniente que presenta es que si un número se quiere enviar para que el codigo de hamming funcione se debe enviar en codigo binario, es decir, no sirve de nada si se envía un número decimal y luego se convierte a binario, es necesario enviar el número en binario completo con dígito de comprobación.

### Comparación

En esta parte se usará el algoritmo de lunh y el codigo de hamming para generar los digitos de comprobación de dos códigos, y se hacen las pruebas que se saben fallan en el algoritmo de luhn.

In [101]:
"""
Codigo que hace una comparación entre los métodos
"""
codigo  = 86679035114
codigo2 = 86670935114



for i in range(10):
    for j in range(10): # for que generan numeros de 0 a 9
        
        #Esta sección solo se ejecuta si el vector código generado es el mismo, al permutar todos los posibles 
        #valores adyacentes entre dos digitos
        
        
        if codigoVer(str(i)+str(j)+"86679035114") == codigoVer(str(j)+str(i)+"86679035114") and i != j:
            
            print("Falla conprobacion algoritmo luhn")
            
            # permuta los primeros 2 elementos del código de manera que se intercambién dos digitos adyacentes
            print("")
            print(str(i)+"----"+str(j))
            print(codigoVer(str(i)+str(j)+"86679035114"))
            print(str(j)+"----"+str(i))
            print(codigoVer(str(j)+str(i)+"86679035114"))
            
            print("")
            print("")
            
            
            
        #Esta sección solo se ejecuta si el vector código generado es el mismo, al permutar todos los posibles 
        #valores adyacentes entre dos digitos
        
        if codigohamming(str(i)+str(j)+"86679035114") == codigohamming(str(j)+str(i)+"86679035114") and i != j:
            
            print("Falla conprobacion codigo de hamming")
            
            # permuta los primeros 2 elementos del código de manera que se intercambién dos digitos adyacentes
            print(str(i)+"----"+str(j))
            print(codigohamming(str(i)+str(j)+"86679035114"))
            print(str(j)+"----"+str(i))
            print(codigohamming(str(j)+str(i)+"86679035114"))
            print("")
            print("")


Falla conprobacion algoritmo luhn

0----5
El digito de control es: 7
5----0
El digito de control es: 7


Falla conprobacion algoritmo luhn

1----6
El digito de control es: 3
6----1
El digito de control es: 3


Falla conprobacion algoritmo luhn

2----7
El digito de control es: 9
7----2
El digito de control es: 9


Falla conprobacion algoritmo luhn

3----8
El digito de control es: 5
8----3
El digito de control es: 5


Falla conprobacion algoritmo luhn

4----9
El digito de control es: 1
9----4
El digito de control es: 1


Falla conprobacion algoritmo luhn

5----0
El digito de control es: 7
0----5
El digito de control es: 7


Falla conprobacion algoritmo luhn

6----1
El digito de control es: 3
1----6
El digito de control es: 3


Falla conprobacion algoritmo luhn

7----2
El digito de control es: 9
2----7
El digito de control es: 9


Falla conprobacion algoritmo luhn

8----3
El digito de control es: 5
3----8
El digito de control es: 5


Falla conprobacion algoritmo luhn

9----4
El digito de 

#### Resultado

En el código anterior se puede ver como para ninguna de las posibles permutaciones entre dos digitos el codigo de hamming falla, y sin embargo para el algoritmo de luhn siguen fallando las mismas permutaciones, la lógica del código se basa en comprobar si el digito de comprobación es el mismo para dos codigos en el algoritmo de luhn de los cuales uno se puede obtener del otro trasplaponiendo dos digitos adyacentes lo cual puede generar errores y para el codigo de hamming se comprueba si el vector generado es igual trasponiendo dos digitos adyacentes.

Como se puede observar en el output del codigo de arriba, para varios valores falla el algoritmo de luhn pero para ninguno el codigo de hamming, de esta forma es posible ver como el código de hamming permite solucionar varios errores del algoritmo de luhn, además permite mostrar en caso de que ocurra un error de cambio de dígito único, en que digito ocurrió el error y solucionarlo.

### Conclusiones

+ Se observó que el algoritmo de luhn presenta varios errores a pesar de que es muy usado en la actualidad para la comprobación de códigos de barras y la creación de algunos dígitos de comprobacion de tarjets de credito.
+ Se observó como el codigo de hamming puede solucionar errores de transposición adyacente del algoritmo de luhn
+ Se mostró como el codigo de hamming puede usarse para mejorar el algoritmo de luhn en casos donde simplemente se hace una mala léctura de un dígito.
+ Se observó que uno de los inconvenientes del codigo de hamming es que trabaja con números binarios, es decir, es necesario diseñar un lector de código de barras que lea número en esta base.
+ Se observó que el algorimo de luhn es de un orden menor que el codigo de hamming.



### Referencias

[1] 	D. Poole, Álgebra lineal, Cengage Learning Editores, S.A. de C.V., 2011. 

[2] 	Instituto de Computación - Facultad de Ingeniería - UDELAR, «Notas de Teórico Códigos y Errores,» Instituto de Computación - Facultad de Ingeniería - UDELAR, pp. 3-8, 2012. 

[3] 	KEYENCE, «automatizacion.eu,» 2010. [En línea]. Available: http://automatizacion.eu/images/aut/barcode/barcode-1D.pdf. [Último acceso: 2020].

[4] 	A. Q. GRACIÁN, «La Teoría de Códigos: una introducción a las Matemáticas,» Departamento de Matemáticas, Universidad Autónoma de Madrid, pp. 6-8.

[5] 	INTERTRADING, «¿Qué es un Código de Barras?,» 2003. [En línea]. Available: http://www.intertrading.cl/PDF/Codigo%20de%20Barras.pdf.

[6] 	numerentur, «numerentur,» [En línea]. Available: http://numerentur.org/funciones-resumen-i/.

[7] 	R. A. Escamilla, «Códigos para detección y corrección de errores en comunicaciones digitales,» pp. 53-57, 2004. 
